In [1]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained('../PalmLM-70000-tokenizer')

/home/itani/vnenv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/itani/vnenv/lib/python3.8/site-packages/transformers/utils/generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# Load model directly
from transformers import AutoModelForCausalLM

# tokenizer = AutoTokenizer.from_pretrained("DAMO-NLP-MT/polylm-1.7b")
model = AutoModelForCausalLM.from_pretrained("DAMO-NLP-MT/polylm-1.7b")

/home/itani/vnenv/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


/home/itani/vnenv/lib/python3.8/site-packages/transformers/utils/generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [23]:
from datasets import load_dataset, interleave_datasets

# lang = ['', '', '', '', '', '', '', '', '', '']

train_afaa = load_dataset("castorini/afriberta-corpus", "afaanoromoo", split="train", streaming=True)
test_afaa = load_dataset("castorini/afriberta-corpus", "afaanoromoo", split="test", streaming=True)
train_amh = load_dataset("castorini/afriberta-corpus", "amharic", split="train", streaming=True)
test_amh = load_dataset("castorini/afriberta-corpus", "amharic", split="test", streaming=True)
train_gah = load_dataset("castorini/afriberta-corpus", "gahuza", split="train", streaming=True)
test_gah = load_dataset("castorini/afriberta-corpus", "gahuza", split="test", streaming=True)
train_hau = load_dataset("castorini/afriberta-corpus", "hausa", split="train", streaming=True)
test_hau = load_dataset("castorini/afriberta-corpus", "hausa", split="test", streaming=True)
train_igb = load_dataset("castorini/afriberta-corpus", "igbo", split="train", streaming=True)
test_igb = load_dataset("castorini/afriberta-corpus", "igbo", split="test", streaming=True)
train_som = load_dataset("castorini/afriberta-corpus", "somali", split="train", streaming=True)
test_som = load_dataset("castorini/afriberta-corpus", "somali", split="test", streaming=True)
train_swa = load_dataset("castorini/afriberta-corpus", "swahili", split="train", streaming=True)
test_swa = load_dataset("castorini/afriberta-corpus", "swahili", split="test", streaming=True)
train_tig = load_dataset("castorini/afriberta-corpus", "tigrinya", split="train", streaming=True)
test_tig = load_dataset("castorini/afriberta-corpus", "tigrinya", split="test", streaming=True)
train_yor = load_dataset("castorini/afriberta-corpus", "yoruba", split="train", streaming=True)
test_yor = load_dataset("castorini/afriberta-corpus", "yoruba", split="test", streaming=True)


multilingual_train = interleave_datasets([train_afaa, train_amh, train_gah, train_hau, train_igb, train_som, train_swa, train_tig, train_yor])
multilingual_test = interleave_datasets([test_afaa, test_amh, test_gah, test_hau, test_igb, test_som, test_swa, test_tig, test_yor])


In [25]:
# from datasets import DatasetDict

# raw_datasets = DatasetDict(
#     {
#         "train": multilingual_train, #.shuffle().select(range(10000)),
#         "valid": multilingual_test, #.shuffle().select(range(100))
#     }
# )

# raw_datasets

DatasetDict({
    train: IterableDataset({
        features: ['id', 'text'],
        n_shards: 1
    })
    valid: IterableDataset({
        features: ['id', 'text'],
        n_shards: 1
    })
})

In [6]:
# for key in raw_datasets["train"][0]:
#     print(f"{key.upper()}: {raw_datasets['train'][0][key][:200]}")

ID: 99045
TEXT: Ihe omuma banyere Gris na ndi Grik.



In [7]:



# outputs = tokenizer(
#     raw_datasets["train"][:2]["text"],
#     truncation=True,
#     max_length=context_length,
#     return_overflowing_tokens=True,
#     return_length=True,
# )

# print(f"Input IDs length: {len(outputs['input_ids'])}")
# print(f"Input chunk lengths: {(outputs['length'])}")
# print(f"Chunk mapping: {outputs['overflow_to_sample_mapping']}")

Input IDs length: 2
Input chunk lengths: [38, 105]
Chunk mapping: [0, 1]


In [39]:
context_length = 128

def tokenize(element):
    outputs = tokenizer(
        element["text"],
        truncation=True,
        max_length=context_length,
        return_overflowing_tokens=True,
        return_length=True,
        padding='max_length',
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
        if length == context_length:
            input_batch.append(input_ids)
    return {"input_ids": input_batch}


tokenized_train = multilingual_train.map(
    tokenize, batched=True, remove_columns=multilingual_train.column_names
)
tokenized_test = multilingual_test.map(
    tokenize, batched=True, remove_columns=multilingual_train.column_names
)

In [40]:
from transformers import DataCollatorForLanguageModeling

tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False)

In [11]:
# out = data_collator([tokenized_datasets["train"][i] for i in range(5)])
# for key in out:
#     print(f"{key} shape: {out[key].shape}")

You're using a LlamaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


input_ids shape: torch.Size([5, 128])
attention_mask shape: torch.Size([5, 128])
labels shape: torch.Size([5, 128])


In [42]:
from transformers import Trainer, TrainingArguments

args = TrainingArguments(
    output_dir="../AfriPalmLM",
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    evaluation_strategy="steps",
    eval_steps=5_000,
    logging_steps=5_000,
    gradient_accumulation_steps=8,
    num_train_epochs=1,
    weight_decay=0.1,
    warmup_steps=1_000,
    lr_scheduler_type="cosine",
    learning_rate=5e-4,
    save_steps=5_000,
    fp16=True,
    push_to_hub=True,
    max_steps=460000 # Because we are streaming, if not it gives errors
)

trainer = Trainer(
    model=model,
    tokenizer=tokenizer,
    args=args,
    data_collator=data_collator,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_test,
)

In [43]:
trainer.train()

OutOfMemoryError: CUDA out of memory. Tried to allocate 64.00 MiB. GPU 0 has a total capacity of 21.99 GiB of which 4.69 MiB is free. Including non-PyTorch memory, this process has 21.98 GiB memory in use. Of the allocated memory 20.90 GiB is allocated by PyTorch, and 4.63 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [22]:
import torch, gc
gc.collect()
torch.cuda.empty_cache()